In [ ]:
import pandas as pd

#Load dataset
df = pd.read_csv("/Users/pranavdalvi/Research Analyst ischool/Task_03_Descriptive_Stats/2024_tw_posts_president_scored_anon.csv")  # Replace with your actual file path

# Convert TRUE/FALSE strings to 1/0 integers
bool_cols = ["isReply", "isRetweet", "isQuote", "isConversationControlled"]

for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.upper().map({"TRUE": 1, "FALSE": 0}).fillna(0).astype(int)

# Drop quoteId and inReplyToId if they exist
df = df.drop(columns=[col for col in ["quoteId", "inReplyToId"] if col in df.columns])

# Save cleaned CSV
df.to_csv("twitter_posts_cleaned.csv", index=False)
print("✅ Cleaned file saved as 'twitter_posts_cleaned.csv'")


✅ Cleaned file saved as 'twitter_posts_cleaned.csv'


In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("/Users/pranavdalvi/Research Analyst ischool/Task_03_Descriptive_Stats/2024_fb_posts_president_scored_anon.csv")

# Remove rows with blank values in key columns
df = df[df["Page Category"].notna() & (df["Page Category"].astype(str).str.strip() != "")]
df = df[df["Page Admin Top Country"].notna() & (df["Page Admin Top Country"].astype(str).str.strip() != "")]

# Drop unwanted columns
columns_to_drop = [
    "Sponsor Id", "Sponsor Name",
    "Sponsor Category", "illuminating_scored_messageelection_integrity_Truth_illuminating"
]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Convert 'Is Video Owner?' column values
df["Is Video Owner?"] = df["Is Video Owner?"].map({
    "Yes": 1,
    "No": 0,
    "-": None
})

# One-hot encode 'Video Share Status' into is_crosspost, is_owned, is_share
video_status_types = ["crosspost", "owned", "share"]
df["Video Share Status"] = df["Video Share Status"].fillna("").str.lower()

for status in video_status_types:
    df[f"is_{status}"] = (df["Video Share Status"] == status).astype(int)

df = df.drop(columns=["Video Share Status"])

# Save cleaned data
df.to_csv("facebook_posts_cleaned.csv", index=False)
print("✅ Cleaned Facebook Posts saved as 'facebook_posts_cleaned.csv'")


✅ Cleaned Facebook Posts saved as 'facebook_posts_cleaned.csv'


In [ ]:
import pandas as pd
import ast

# Load dataset
df = pd.read_csv("/Users/pranavdalvi/Research Analyst ischool/Task_03_Descriptive_Stats/2024_fb_ads_president_scored_anon.csv")  # Adjust path

# Convert demographic_distribution column from string to dict
df["demographic_distribution"] = df["demographic_distribution"].apply(
    lambda x: ast.literal_eval(x) if pd.notnull(x) and isinstance(x, str) else {}
)

# Step 2: Flatten demographic_distribution into a long-form list of records
records = []
for idx, row in df.iterrows():
    ad_id = row.get("ad_id") or row.get("Facebook_Id") or row.get("id")  # Use whichever column exists
    demographics = row["demographic_distribution"]

    if isinstance(demographics, dict):
        for key, values in demographics.items():
            if isinstance(values, dict):
                try:
                    gender, age = key.split("_")
                    records.append({
                        "ad_id": ad_id,
                        "gender": gender,
                        "age_group": age,
                        "spend": values.get("spend", 0),
                        "impressions": values.get("impressions", 0)
                    })
                except ValueError:
                    continue  # skip malformed keys

# Create the long-form demographics dataframe
df_demo_long = pd.DataFrame(records)

# Drop the original demographic_distribution column from the main df
df = df.drop(columns=["demographic_distribution"])

df_merged = df.merge(df_demo_long, on="ad_id", how="inner")

# Save the final merged dataset
df_merged.to_csv("ads_with_demographics_expanded.csv", index=False)
print("✅ Final dataset saved as 'ads_with_demographics_expanded.csv'")


✅ Final dataset saved as 'ads_with_demographics_expanded.csv'


In [ ]:
import pandas as pd
import ast

# Load original dataset
df = pd.read_csv("/Users/pranavdalvi/Research Analyst ischool/Task_03_Descriptive_Stats/2024_fb_ads_president_scored_anon.csv")

# Convert demographic_distribution column from string to dict
df["demographic_distribution"] = df["demographic_distribution"].apply(
    lambda x: ast.literal_eval(x) if pd.notnull(x) and isinstance(x, str) else {}
)

# Flatten demographic_distribution into a long-form list of records
records = []
for idx, row in df.iterrows():
    ad_id = row.get("ad_id") or row.get("Facebook_Id") or row.get("id")  # Use whichever column exists
    demographics = row["demographic_distribution"]

    if isinstance(demographics, dict):
        for key, values in demographics.items():
            if isinstance(values, dict):
                try:
                    gender, age = key.split("_")
                    records.append({
                        "ad_id": ad_id,
                        "gender": gender,
                        "age_group": age,
                        "spend": values.get("spend", 0),
                        "impressions": values.get("impressions", 0)
                    })
                except ValueError:
                    continue  # skip malformed keys

# Create the long-form demographics dataframe
df_demo_long = pd.DataFrame(records)

# Convert gender to numeric values: male=1, female=0, unknown=None
df_demo_long["gender"] = df_demo_long["gender"].map({
    "male": 1,
    "female": 0,
    "unknown": None
})

# Drop the original demographic_distribution column from the main df
df = df.drop(columns=["demographic_distribution"])

# Merge long-form demographics with main dataframe
df_merged = df.merge(df_demo_long, on="ad_id", how="inner")

# Save the final merged dataset
df_merged.to_csv("ads_with_demographics_expanded.csv", index=False)
print("✅ Final dataset saved as 'ads_with_demographics_expanded.csv'")


✅ Final dataset saved as 'ads_with_demographics_expanded.csv'
